<a href="https://colab.research.google.com/github/dalgual/aidatasci/blob/main/opentable_sentiment_helpful.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
!pip install torch
!pip install tensorflow

In [139]:
!pip install accelerate -U

In [140]:
!pip install transformers[torch]

In [141]:
import torch
torch.cuda.is_available()

False

In [142]:
!pip install datasets transformers huggingface_hub


In [143]:
# !apt-get install git-lfs


In [144]:
from huggingface_hub import notebook_login
notebook_login()


In [145]:
import pandas as pd

data_files = "https://raw.githubusercontent.com/dalgual/aidatasci/master/data/classification/opentable_review.csv"
#pd.options.display.max_rows = 9 #999

df = pd.read_csv(data_files) #'data.csv')

#print(df)

In [146]:
# Drop null data
df = df.dropna(subset=['content'])

In [147]:
df.info(verbose=True)

In [148]:
import numpy as np

#define conditions
conditions = [
    df['helpful']==0,
    df['helpful']>0
]

#define results
results = [0, 1]

#create new column based on conditions in column1 and column2
df['labels'] = np.select(conditions, results)


In [184]:
df

,Unnamed: 0,nick,score,content,date,helpful,labels
0,45435,A Google user,5,I just found and reserved a table for dinner t...,2009-09-14 17:21:18,0,0
1,45541,A Google user,3,Good start takes up too much space and leaves ...,2009-09-14 21:00:11,0,0
2,45925,A Google user,5,Koo app . Very useable,2009-09-15 3:28:32,0,0
3,45446,A Google user,4,"Awesome, just waaaaaayyyyy too big for this ki...",2009-09-15 15:43:21,0,0
...,...,...,...,...,...,...,...
46388,170,Kent Weber,4,Very useful app when exploring a new area. Jus...,2023-01-15 23:57:52,0,0
46389,3525,Tim Bolognone,5,Easy to use.,2023-01-16 1:02:59,0,0
46390,238,Neil Stout,4,Easy to determine availability and to book a r...,2023-01-16 2:36:28,0,0
46391,15023,Taeyang Kim,5,Good,2023-01-16 5:27:46,0,0


In [185]:
df[df['labels'] == 1]

,Unnamed: 0,nick,score,content,date,helpful,labels
232,45693,A Google user,5,"Cool app., needs better search capabilities. ....",2010-05-18 2:09:24,1,1
264,44832,A Google user,4,Love Open Table and this is a good app. Add th...,2010-06-30 2:24:48,1,1
324,44664,A Google user,5,Great idea. Booked a reservation for my annive...,2010-08-12 17:47:12,1,1
325,44542,A Google user,3,The menu doesn't scroll correctly on my HTC In...,2010-08-12 22:01:33,1,1
...,...,...,...,...,...,...,...
46338,14992,A Google user,1,This app is *still* so full of fail. How is th...,2023-01-12 4:06:59,1,1
46357,99,Tim,4,Great for finding and reserving restaurants al...,2023-01-13 23:12:02,1,1
46373,78,D. Coven,3,I would say is that most restaurants are vastl...,2023-01-14 20:51:14,3,1
46382,2,Laurie Gorecki,1,"I've used the app for years, and in the past g...",2023-01-15 15:49:02,25,1


In [186]:
df1 = df.sample(frac = 0.7, random_state = 200)

In [187]:
df1


,Unnamed: 0,nick,score,content,date,helpful,labels
2396,42831,A Google user,5,great app.,2012-02-14 14:50:31,1,1
3443,45090,A Google user,4,"Great app to have, would give it 5 stars if it...",2012-11-05 0:10:38,0,0
24578,21433,A Google user,3,"Not sure how new it is, ive just downloaded it...",2018-01-17 2:05:12,0,0
7237,39219,A Google user,5,Cool!,2015-08-21 22:39:27,0,0
...,...,...,...,...,...,...,...
10299,34976,A Google user,1,Can't make reservations through the app. Only ...,2015-12-25 4:46:43,0,0
19625,26423,A Google user,5,"Extremely handy managing reservations, using i...",2017-03-13 23:21:46,0,0
10432,35205,A Google user,4,Just need to represent more restaurants in my ...,2015-12-30 6:09:29,0,0
13911,31712,A Google user,5,Hate phoning for reservations at a restaurant....,2016-05-10 22:35:25,0,0


In [150]:
from datasets import load_dataset
from datasets import Dataset

# dataset = load_dataset("csv", data_files=data_files, split="train")
dataset = Dataset.from_pandas(df)


In [188]:
dataset

Dataset({
    features: ['Unnamed: 0', 'nick', 'score', 'content', 'date', 'helpful', 'labels', '__index_level_0__'],
    num_rows: 46349
})

In [152]:
# https://discuss.huggingface.co/t/how-to-split-hugging-face-dataset-to-train-and-test/20885/3
train_ds=dataset.train_test_split(test_size=0.3, shuffle=True) #, stratify_by_column='Sentiment')
print(train_ds)

In [153]:
#train_ds = train_ds.rename_column("Sentiment", "labels")
train_ds = train_ds.rename_column("content", "text")
print(train_ds)

In [189]:
train_ds

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'nick', 'score', 'text', 'date', 'helpful', 'labels', '__index_level_0__'],
        num_rows: 32444
    })
    test: Dataset({
        features: ['Unnamed: 0', 'nick', 'score', 'text', 'date', 'helpful', 'labels', '__index_level_0__'],
        num_rows: 13905
    })
})

In [190]:
small_train_dataset = train_ds["train"].shuffle(seed=42).select([i for i in list(range(32444))])
small_test_dataset = train_ds["test"].shuffle(seed=42).select([i for i in list(range(13905))])


In [191]:
small_train_dataset

Dataset({
    features: ['Unnamed: 0', 'nick', 'score', 'text', 'date', 'helpful', 'labels', '__index_level_0__'],
    num_rows: 32444
})

In [192]:
small_test_dataset

Dataset({
    features: ['Unnamed: 0', 'nick', 'score', 'text', 'date', 'helpful', 'labels', '__index_level_0__'],
    num_rows: 13905
})

In [193]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


In [194]:
def preprocess_function(examples):
   #return tokenizer(examples["SentimentText"], truncation=True)
   return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/32444 [00:00<?, ? examples/s]

Map:   0%|          | 0/13905 [00:00<?, ? examples/s]

In [195]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [196]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [197]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
   load_prec = load_metric("precision")
   load_recall = load_metric("recall")
   load_auc = load_metric("roc_auc")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   prec = load_prec.compute(predictions=predictions, references=labels)["precision"]
   recall = load_recall.compute(predictions=predictions, references=labels)["recall"]
   auc = load_auc.compute(predictions=predictions, references=labels)["roc_auc"]

   return {"roc_auc":roc_auc, "accuracy": accuracy, "f1": f1, "precision":prec, "recall":recall}


In [198]:
#!pip install accelerate -U

In [199]:
#!pip install transformers[torch]

In [200]:
from transformers import TrainingArguments, Trainer


#repo_name = "finetuning-sentiment-model-3000-samples-jwoo5"
repo_name = "semadalg/finetuning-opetable-model"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)




In [201]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

In [202]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


Step,Training Loss
500,0.356600
1000,0.325900
1500,0.336000
2000,0.338500
2500,0.325300
3000,0.319900
3500,0.313000
4000,0.320200


TrainOutput(global_step=4056, training_loss=0.3300723554584872, metrics={'train_runtime': 20185.7222, 'train_samples_per_second': 3.215, 'train_steps_per_second': 0.201, 'total_flos': 949079236095744.0, 'train_loss': 0.3300723554584872, 'epoch': 2.0})

In [203]:
trainer.evaluate()

{'eval_loss': 0.3368667662143707,
 'eval_accuracy': 0.8719884933477167,
 'eval_f1': 0.29810725552050477,
 'eval_precision': 0.5264623955431755,
 'eval_recall': 0.2079207920792079,
 'eval_runtime': 1266.3845,
 'eval_samples_per_second': 10.98,
 'eval_steps_per_second': 0.687,
 'epoch': 2.0}

In [ ]:
trainer.push_to_hub()


In [205]:
from transformers import pipeline

sentiment_model = pipeline(model="semadalg/finetuning-opetable-model")
sentiment_model(["I love this move", "This movie sucks!"])


[{'label': 'LABEL_0', 'score': 0.9733502864837646},
 {'label': 'LABEL_0', 'score': 0.9096870422363281}]

In [206]:
predictions = trainer.predict(tokenized_test)
# y_pred = trainer.predict(test_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
predictions.predictions.shape

In [ ]:
predictions.label_ids.shape

In [ ]:
predictions

In [210]:
df_preds=pd.DataFrame(predictions.predictions)
df_preds.apply(lambda row : np.exp(row[0])/(np.exp(row[0])+np.exp(row[1])),axis=1)

0        0.950069
1        0.950724
2        0.952431
3        0.980047
           ...   
13901    0.984457
13902    0.937364
13903    0.854015
13904    0.946891
Length: 13905, dtype: float32

In [211]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)



In [212]:
probs = np.exp(preds[:])

In [213]:
preds

array([0, 0, 0, ..., 0, 0, 0])

In [214]:
preds2 = np.argmax(predictions.predictions, axis=1)

In [215]:
preds2

array([0, 0, 0, ..., 0, 0, 0])

In [216]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids, zero_division=0)

ModuleNotFoundError: ignored

## References
1. Getting Started with Sentiment Analysis using Python, https://huggingface.co/blog/sentiment-analysis-python
1. The model did not return a loss from the inputs, only the following keys: logits, https://discuss.huggingface.co/t/the-model-did-not-return-a-loss-from-the-inputs-only-the-following-keys-logits-for-reference-the-inputs-it-received-are-input-values/25420/3
